# BLS

In [ ]:
!pip install -r requirements.txt

## Fetching and Parsing a BLS RSS Feed

In [ ]:
import feedparser
import datetime

rss_url = "https://www.bls.gov/feed/bls_latest.rss"  # e.g., Latest Numbers feed
feed = feedparser.parse(rss_url)

for entry in feed.entries:
    title = entry.title            # e.g. "Employment Situation"
    published = entry.published    # e.g. "Fri, 09 Jan 2026 08:30:00 -0500"
    link = entry.link              # URL to detailed news release
    summary = entry.summary        # HTML summary of the release
    print(f"Release: {title}, Date: {published}")
    # Here you could compare published date with last seen date to identify new items


## Querying the BLS API for Data

In [ ]:
import requests
import json

series_to_get = ["CUSR0000SA0", "LNS14000000"]  # CPI-U all items, and unemployment rate series IDs
payload = {
    "seriesid": series_to_get,
    "startyear": "2025", 
    "endyear": "2026"
    # You can include your "registrationkey": "YOUR_API_KEY_HERE" if required by v2.
}
response = requests.post("https://api.bls.gov/publicAPI/v2/timeseries/data/", 
                         json=payload, timeout=10)
data = response.json()  # parse JSON response to dict

if data.get("status") == "REQUEST_SUCCEEDED":
    series_list = data["Results"]["series"]
    for series in series_list:
        sid = series["seriesID"]
        print(f"Data for series {sid}:")
        for item in series["data"]:
            year = item["year"]; period = item["period"]; value = item["value"]
            print(f" {year}-{period}: {value}")
else:
    print("API request failed or returned an error:", data.get("message"))


## Scheduling the Polling Function

In [ ]:
import schedule
import time

def poll_bls():
    print("Polling BLS for updates...")
    # Here you would call the RSS fetching function and/or API fetching function.
    # e.g., check feeds, download new data, save to files.
    # For demonstration, just printing timestamp.
    import datetime; print("Polled at", datetime.datetime.now())

# Schedule the poll_bls job every 6 hours
schedule.every(6).hours.do(poll_bls)

# Keep the script running to execute scheduled tasks
while True:
    schedule.run_pending()
    time.sleep(60)  # sleep a bit between checks


## Storing Data to CSV

In [ ]:
import csv

new_data = {"series": "Unemployment Rate", "date": "2025-12", "value": 4.4}
csv_file = "unemployment_rate.csv"

# If file does not exist, write header
import os
file_exists = os.path.isfile(csv_file)
with open(csv_file, mode='a', newline='') as f:
    writer = csv.writer(f)
    if not file_exists:
        writer.writerow(["series", "date", "value"])
    writer.writerow([new_data["series"], new_data["date"], new_data["value"]])


## CLI Argument Parsing (using argparse)

In [ ]:
import argparse

parser = argparse.ArgumentParser(description="BLS Data Polling CLI")
parser.add_argument("--once", action="store_true", help="Run a single poll and exit")
parser.add_argument("--use-api", action="store_true", help="Fetch data via API calls (default uses RSS triggers)")
args = parser.parse_args()

if args.once:
    # Run one iteration of polling
    poll_bls()  # assume this function encapsulates one full poll cycle
else:
    # Run continuous schedule
    if args.use_api:
        configure_mode(api_only=True)
    start_scheduler()  # assume this sets up schedule as above
